In [1]:
import os
import pandas as pd
import numpy as np
import pyspark

import pyscisci.all as pyscisci
import time

path2mag = '/users/hgt6rn/Documents/DataSets/MAG'

mymag = pyscisci.MAG(path2mag, database_extension='hdf', show_progress=True)

In [2]:
pub2year = mymag.pub2year
pub2000 = np.sort([pid for pid,y in pub2year.items() if y==2000])
print(pub2000.shape)

(4279539,)


In [3]:
pub2ref = mymag.load_references(filter_dict={'CitedPublicationId':pub2000})


Loading pub2ref:   0%|          | 0/186 [00:00<?, ?it/s]

In [4]:
stime = time.time()

ncites = pub2ref.groupby('CitedPublicationId')['CitingPublicationId'].nunique()

etime = time.time()
print(etime-stime)

37.078683853149414


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct

spark = SparkSession.builder.config("spark.driver.memory", "50g").getOrCreate()

stime = time.time()
df = spark.createDataFrame(pub2ref)
print("Create df:", time.time()-stime)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/09 14:49:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/opt/homebrew/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/homebrew/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [ ]:
stime = time.time()

df.groupby('CitedPublicationId').agg(countDistinct('CitingPublicationId')).show()

print("Compute Groupby:", time.time()-stime)